論文  
https://arxiv.org/abs/2101.02702v2  
  
GitHub  
https://github.com/timmeinhardt/trackformer  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/TrackFormer_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# GPU確認

In [ ]:
!nvidia-smi

# 環境セットアップ

## GitHubからTrackFormerのソースコードを取得

In [ ]:
%cd /content/

!git clone https://github.com/timmeinhardt/trackformer.git

## ライブラリのインストール

In [ ]:
%cd /content/trackformer

!pip install -r requirements.txt

!pip uninstall torch -y
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

!pip install -U 'git+https://github.com/timmeinhardt/cocoapi.git#subdirectory=PythonAPI'
!python src/trackformer/models/ops/setup.py build --build-base=src/trackformer/models/ops/ install

## バージョン変更したPytorchの動作確認

In [ ]:
!python -c "import torch; print(torch.__version__); print(torch.cuda.is_available()); print(torch.cuda.device_count());"

# 事前学習済みモデルのダウンロード

In [ ]:
%cd /content/trackformer/models

!wget https://vision.in.tum.de/webshare/u/meinhard/trackformer_models.zip
!unzip trackformer_models.zip

# Multi-object tracking

In [ ]:
%cd /content/trackformer

!ffmpeg -i data/snakeboard/snakeboard.mp4 -vf fps=30 data/snakeboard/%06d.png

In [ ]:
%%time

!python src/track.py with \
  dataset_name=DEMO \
  data_root_dir=data/snakeboard \
  output_dir=data/snakeboard \
  write_images=pretty

## トラッキング結果を動画に変換



In [ ]:
%cd /content/trackformer

results_video = "data/snakeboard/DEMO/snakeboard_result.mp4"

!ffmpeg -i data/snakeboard/DEMO/snakeboard/%06d.png -c:v libx264 -vf "fps=25,format=yuv420p" {results_video}

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(results_video, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="60%" height="60%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# アップロードした動画のMulti-object tracking
使用動画  
https://www.pexels.com/ja-jp/video/3873059/  
https://www.pexels.com/ja-jp/video/853889/  

In [ ]:
%cd /content/trackformer/data
!rm -rf upload
!mkdir -p upload
%cd upload

from google.colab import files

uploaded = files.upload()
uploaded = list(uploaded.keys())
file_name = uploaded[0]

print(file_name)

In [ ]:
# 半角スペース除去
rename = file_name.replace(" ", "")

import os
os.rename(file_name, rename)

## 動画をフレーム画像に分割

In [ ]:
!ffmpeg -i {rename} ./%06d.png

## Multi-object tracking実施

In [ ]:
%%time

%cd /content/trackformer

!python src/track.py with \
  dataset_name=DEMO \
  data_root_dir=data/upload \
  output_dir=data/upload \
  write_images=pretty

## トラッキング結果を動画に変換

In [ ]:
%cd /content/trackformer

results_video = "data/upload/DEMO/upload_result.mp4"

!ffmpeg -i data/upload/DEMO/upload/%06d.png -c:v libx264 -vf "fps=25,format=yuv420p" {results_video}

In [ ]:
mp4 = open(results_video, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="60%" height="60%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# Multi-object tracking(Segmentation)

In [ ]:
%cd /content/trackformer/data
!rm -rf upload_seg
!mkdir -p upload_seg
%cd upload_seg

from google.colab import files

uploaded_seg = files.upload()
uploaded_seg = list(uploaded_seg.keys())
file_name = uploaded_seg[0]

print(file_name)

In [ ]:
# 半角スペース除去
rename = file_name.replace(" ", "")

import os
os.rename(file_name, rename)

In [ ]:
!ffmpeg -i {rename} ./%06d.png

In [ ]:
%%time

%cd /content/trackformer

!python src/track.py with \
  dataset_name=DEMO \
  obj_detect_checkpoint_file=models/mots20_train_masks/checkpoint.pth \
  data_root_dir=data/upload_seg \
  output_dir=data/upload_seg \
  write_images=pretty

In [ ]:
%cd /content/trackformer

results_video = "data/upload_seg/DEMO/upload_seg_result.mp4"

!ffmpeg -i data/upload_seg/DEMO/upload_seg/%06d.png -c:v libx264 -vf "fps=25,format=yuv420p" {results_video}

In [ ]:
mp4 = open(results_video, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="60%" height="60%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")